In [33]:
# default_exp validation

In [34]:
# hide
%reload_ext lab_black

# Geometry Validation

> geometry validations

In [35]:
# exporti
import logging
from abc import ABC, abstractmethod
from typing import Sequence, Union

import geopandas as gpd
from fastcore.basics import patch
from shapely.algorithms.cga import signed_area
from shapely.geometry.base import BaseGeometry
from shapely.geometry.polygon import orient

logger = logging.getLogger(__name__)

In [41]:
# export
class ValidationError(Exception):
    pass

In [42]:
# export
class Validator(ABC):
    """Abstract Base Class for single validator"""

    def __init__(
        self,
        add_new_column: bool = True,
        apply_fix: bool = True,
    ):
        self.add_new_column = add_new_column
        self.apply_fix = apply_fix

    @property
    @abstractmethod
    def validator_column_name(self):  # pragma: no cover
        pass

    def check(self, geometry: BaseGeometry):  # pragma: no cover
        pass

    def fix(self, geometry: BaseGeometry):  # pragma: no cover
        pass


@patch
def validate(self: Validator, gdf: gpd.GeoDataFrame, clone=True) -> gpd.GeoDataFrame:
    """Method that checks the validity of a each geometry and applies a fix to these geometry"""
    if clone:
        gdf = gdf.copy()
    is_valid = gdf.geometry.apply(self.check)
    if self.add_new_column:
        gdf[self.validator_column_name] = is_valid
    if self.apply_fix:
        gdf.loc[~is_valid, "geometry"] = gdf[~is_valid].geometry.apply(self.fix)
    return gdf

In [43]:
# export


class OrientationValidator(Validator):
    """Checks and fixes Orienation of the geometry to ensure it works for multiple system"""

    validator_column_name = "is_oriented_properly"


@patch
def check(self: OrientationValidator, geometry: BaseGeometry) -> bool:
    """Checks if orientation is counter clockwise"""
    return signed_area(geometry.exterior) >= 0


@patch
def fix(self: OrientationValidator, geometry: BaseGeometry) -> BaseGeometry:
    """Fixes orientation if orientation is clockwise"""
    return orient(geometry)

In [44]:
# export
class GeometryValidation:
    """Applies a list of validation checks and tries to fix them"""

    validators_map = {"orientation": OrientationValidator}

    def __init__(
        self,
        gdf: gpd.GeoDataFrame,
        validators: Sequence[Union[str, Validator]] = ("orientation",),
        add_validation_columns: bool = True,
        apply_fixes: bool = True,
    ) -> gpd.GeoDataFrame:
        self.gdf = gdf
        self.validators = validators
        self.add_validation_columns = add_validation_columns
        self.apply_fixes = apply_fixes

    def _get_validators(self) -> Sequence[Validator]:
        """Gets a list of Validator Classes based on string"""
        validators_classes = []
        for validator in self.validators:
            if isinstance(validator, str):
                if validator not in self.validators_map:
                    raise ValidationError("Invalid validator.")
                validator = self.validators_map[validator]
                validators_classes.append(validator)
            else:
                raise ValidationError("Invalid validator.")
        return validators_classes


@patch
def validate_all(self: GeometryValidation) -> gpd.GeoDataFrame:
    """Sequentially run validators"""
    validators = self._get_validators()
    gdf = self.gdf
    for validator in validators:
        gdf = validator(
            add_new_column=self.add_validation_columns,
            apply_fix=self.apply_fixes,
        ).validate(gdf)
    return gdf

In [45]:
# hide
from nbdev.export import notebook2script

notebook2script("00_validation.ipynb")

Converted 00_validation.ipynb.
